In [1]:
import mxnet as mx
from mxnet import gluon
from mxnet.gluon import nn
from mxnet import autograd as ag
from mxnet import ndarray as nd
from Algorithm.CNN import CNN_Model
from Algorithm.MLP import MLP
from Tools import utils
LeNet = CNN_Model('LeNet')
ctx = utils.try_all_gpus()
LeNet.initialize(init=mx.initializer.Xavier(),ctx=ctx)

In [4]:
import numpy as np
train_data = np.load("train_data0.npy")
train_label = np.load("train_label0.npy")
batch_size = 100
train_data = mx.io.NDArrayIter(train_data, train_label, batch_size, shuffle=True)

In [5]:
def train(net, data, epoch):
    trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.02})
    metric = mx.metric.Accuracy()
    softmax_cross_entropy_loss = gluon.loss.SoftmaxCrossEntropyLoss()
    for i in range(epoch):
        train_data.reset()
        for batch in train_data:
            data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
            label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
            outputs = []
            with ag.record():
                for x, y in zip(data, label):
                    z = net(x)
                    loss = softmax_cross_entropy_loss(z, y)
                    loss.backward()
                    outputs.append(z)
            metric.update(label, outputs)
            trainer.step(batch.data[0].shape[0])
        name, acc = metric.get()
        metric.reset()
        print('training acc at epoch %d: %s=%f'%(i, name, acc))

In [6]:
train(LeNet, train_data, 10)

training acc at epoch 0: accuracy=0.088333
training acc at epoch 1: accuracy=0.090000
training acc at epoch 2: accuracy=0.091667
training acc at epoch 3: accuracy=0.096667
training acc at epoch 4: accuracy=0.120000
training acc at epoch 5: accuracy=0.176667
training acc at epoch 6: accuracy=0.236667
training acc at epoch 7: accuracy=0.285000
training acc at epoch 8: accuracy=0.301667
training acc at epoch 9: accuracy=0.368333
